In [1]:
import pandas as pd
import numpy as np
import custom_transformers

In [3]:
import gensim.downloader
glove_twitter_25_model = gensim.downloader.load('glove-twitter-200')
glove_twitter_25_vocab = glove_twitter_25_model.key_to_index.keys()
df = pd.read_csv('data/clean_data.csv',keep_default_na=False)

In [41]:
'answer' in df.columns

True

In [34]:
vocab = glove_twitter_25_model.index_to_key

In [40]:
df['clue'].iloc[0].isin(vocab)

AttributeError: 'str' object has no attribute 'isin'

In [25]:
glove_twitter_25_model.get_vector('litigator')

KeyError: "Key 'litigator' not present"

In [23]:
glove_twitter_25_model[df['clue'].iloc[0].split()]

KeyError: "Key 'litigator' not present"

In [12]:
Goodbye

array([ 1.8970e-01, -8.7202e-01, -1.6335e-01, -2.3767e-01, -7.1186e-01,
        5.7040e-01,  5.0581e-01,  3.0969e-01, -4.0642e-01,  5.2703e-02,
        1.0854e-01, -1.6150e-01, -3.4698e-01,  6.1637e-02, -7.6703e-02,
       -5.1286e-01, -1.0722e-01, -6.8998e-03,  2.2470e-01,  1.8388e-01,
       -2.9473e-01, -1.8634e-01, -2.6968e-02,  3.8135e-01,  1.2956e-01,
        6.3070e-01,  1.8566e-01, -2.3255e-01,  1.5142e-01, -3.1132e-01,
       -2.8968e-01, -2.8378e-01, -9.6991e-02,  3.3150e-01, -2.2192e-01,
       -2.2665e-01, -6.6048e-01, -2.1500e-01,  9.5876e-01,  8.6441e-01,
        1.3507e-01,  3.8631e-01, -7.3043e-02, -1.8944e-01, -2.2348e-01,
       -1.5787e-04,  6.2617e-02, -1.7597e-01,  3.1616e-01,  1.8281e-01,
        1.3680e-01,  8.5130e-02, -1.5666e-01, -1.2542e-01,  9.1975e-02,
        5.5632e-01,  1.1411e-03,  8.4846e-01, -3.5636e-01, -2.3754e-01,
        1.1621e+00,  6.7459e-02, -3.7497e-01, -1.8960e-01, -2.7032e-01,
       -2.8898e-01, -1.8276e-01,  2.6633e-01,  3.2742e-02, -1.93

In [15]:
df.head(10)

,answer,clue,noun_involved,fill_blank
0,aba,litigator group,False,False
1,actor,thespian,False,False
2,ades,summer coolers,False,False
3,ads,newspaper revenue,False,False
4,afro,head of hair,False,False
5,agate,quartz,False,False
6,amen,famous last word,False,False
7,amir,top turk,True,False
8,anise,cordial flavoring,False,False
9,ans,part of briefly,True,False


In [40]:
words = [x[0] for x in glove_twitter_25_model.similar_by_word('thespian', topn=1000)]
words

['chronograph',
 'amethyst',
 'analog',
 'gemstone',
 'stainless',
 'bezel',
 'bracelet',
 'topaz',
 'citrine',
 'faceted',
 'swarovski',
 'crystal',
 'aquamarine',
 'beaded',
 'resin',
 'plated',
 'dial',
 'crystals',
 'copper',
 'turquoise',
 'accented',
 'agate',
 'silver',
 'steel',
 'swiss',
 'pearl',
 'bead',
 'textured',
 'ceramic',
 'necklace',
 'rhinestone',
 'two-tone',
 'wristwatch',
 'earrings',
 'multi-function',
 'mother-of-pearl',
 'chrono',
 'cubic',
 'strap',
 'druzy',
 'sterling',
 'zirconia',
 'unisex',
 'womens',
 'bangle',
 'smoky',
 'antiqued',
 'metallic',
 'beads',
 'charm',
 'peridot',
 'freshwater',
 'oval',
 'brass',
 'adjustable',
 'simulated',
 'alloy',
 'jewelry',
 'stainless-steel',
 'inlay',
 'clasp',
 'wrist',
 'diamond',
 'pendants',
 'mens',
 'pendant',
 'emerald',
 'invicta',
 'gunmetal',
 'lattice',
 'aventurine',
 'handcrafted',
 'gold-plated',
 'fossil',
 'silicone',
 'garnet',
 'thermal',
 'leather',
 'jeweled',
 'tanzanite',
 'silver-tone',
 'ba

In [85]:
clue = np.mean(glove_twitter_25_model['top','turk'],axis=0)

In [86]:
answer = glove_twitter_25_model.get_vector('amir')

In [87]:
glove_twitter_25_model.similar_by_vector(clue)

[('top', 0.7888505458831787),
 ('turk', 0.7607848644256592),
 ('star', 0.5923318266868591),
 ('best', 0.5916703939437866),
 ('the', 0.5785344243049622),
 ('of', 0.5733984112739563),
 ("'s", 0.5634052753448486),
 ('in', 0.5548969507217407),
 ('one', 0.5532031059265137),
 ('is', 0.5445180535316467)]

In [89]:
cosine_similarity([clue, answer])

array([[1.0000001 , 0.16243619],
       [0.16243619, 1.0000002 ]], dtype=float32)

In [82]:
glove_twitter_25_model.similarity('turk','amir')

0.10567921

In [90]:
glove_twitter_25_model.similarity('top','amir')

0.14508834

In [79]:
glove_twitter_25_model.similarity(clue, answer)

ValueError: shapes (1,200) and (1,200) not aligned: 200 (dim 1) != 1 (dim 0)

In [56]:
glove_twitter_25_model['top','turk'].shape

(2, 200)

In [58]:
glove_twitter_25_model.most_similar('london')

[('uk', 0.7124235033988953),
 ('dublin', 0.7085306644439697),
 ('birmingham', 0.7076107859611511),
 ('sydney', 0.6874022483825684),
 ('glasgow', 0.6842960119247437),
 ('nyc', 0.6601422429084778),
 ('york', 0.6552266478538513),
 ('town', 0.6535198092460632),
 ('melbourne', 0.647788941860199),
 ('edinburgh', 0.6421033143997192)]

In [62]:
glove_twitter_25_model.similar_by_vector(np.mean(glove_twitter_25_model['top','turk'],axis=0))

[('top', 0.7888505458831787),
 ('turk', 0.7607848644256592),
 ('star', 0.5923318266868591),
 ('best', 0.5916703939437866),
 ('the', 0.5785344243049622),
 ('of', 0.5733984112739563),
 ("'s", 0.5634052753448486),
 ('in', 0.5548969507217407),
 ('one', 0.5532031059265137),
 ('is', 0.5445180535316467)]